# Autocorrélation spatiale
Reproduction avec R l'essentiel d'une analyse menée avec Geoda.

-/-/- - Author: Abdoulaye DIALLO  
-/-/- - École: ENSAI  
-/-/- - Cours: Spatial Data  

#### Sources et Références
#### [Geospatial Data Analysis | SciPy 2018 | Serge Rey](https://www.youtube.com/watch?v=kJXUUO5M4ok)  
#### [--](--)
#### [--](--)  

----------------------------

La première loi de la géographie de **Waldo Tober**: `tout est lié à tout le reste, mais les choses proches sont plus liées que les choses lointaine`.

----------------------------

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point

KeyError: 'MISSING'

## Les donnnées

#### chargement des données non spatial

In [4]:
# Chargeons les données non spatial à fusionée avec les données spati
medianhousing = gpd.read_file("medianhousing/medianhousing.dbf")
print(medianhousing.crs)

epsg:3857


In [7]:
medianhousing.head(2)

,POLY_ID,GEO_ID,STATE,COUNTY,TRACT,NAME,LSAD,SHAPE_AREA,SHAPE_LEN,Id,Id2,Geography,Median_val,Median_v_1,geometry
0,1,1400000US25017300100,25,017,300100,3001,Tract,1.153484e+08,48105.939998,1400000US25017300100,2.501730e+10,"Census Tract 3001, Middlesex County, Massachus...",286700.0,15865.0,"POLYGON ((-7989680.091 5267632.891, -7989885.5..."
1,2,1400000US25017301101,25,017,301101,3011.01,Tract,8.252731e+07,41749.428647,1400000US25017301101,2.501730e+10,"Census Tract 3011.01, Middlesex County, Massac...",290800.0,14919.0,"POLYGON ((-7989680.091 5267632.891, -7989536.8..."


In [10]:
import pysal
import numpy as np
np.random.seed(12345)